In [1]:
# from mido import MidiFile, MidiTrack, Message
import mido
print(f"\nMidosVersion: {mido.__version__}")
import numpy as np 
from matplotlib import pyplot as plt
import os


MidosVersion: 1.2.9


In [3]:
def midtest(data_folders):
    
    all_mid = []
    ignored = 0
    for dirpath, _, filenames in os.walk(data_folders):
        for File in filenames:
            path = os.path.join(dirpath, File)
            if not (path.endswith('.mid') or path.endswith('.midi')):
                print(f"{path} ignored")
                ignored +=1
                continue
            else:
                all_mid.append(mido.MidiFile(path))
                    
    return all_mid

In [4]:
mid = midtest("dataset")

In [5]:
for music in mid:
    for track in music.tracks:
        for msg in track:
            
            if msg.is_meta:
                continue
    
            if msg.channel not in [0,1,2]:
                try:
                    mid.remove(music)
                except ValueError:
                    continue

In [6]:
mid[0]

<midi file 'dataset\\Animal Crossing - 100 PM.mid' type 1, 2 tracks, 1167 messages>

In [7]:
def midi_to_samples(fname, SPM):
    mid = fname
#     mid = mido.MidiFile(fname)
    TPM = mid.ticks_per_beat

    for track in mid.tracks:
        for msg in track:
            if msg.type == 'time_signature':
                new_tpm = msg.denominator * TPM/msg.numerator

    all_notes = {}

    for track in mid.tracks:
        abs_time = 0
        for msg in track:
            abs_time += msg.time
            
            #Notes starts
            if msg.type == "note_on":
                #Skip notes that we can´t hear
                if msg.velocity == 0:
                    continue
                
                note = msg.note
                
                if note not in all_notes:
                    all_notes[note] = []
                    
                else:
                    single_note = all_notes[note][-1] # If already note_on then end that note
                    if len(single_note) == 1:
                        single_note.append(single_note[0] + 1) 
                        
                all_notes[note].append([abs_time * SPM / TPM]) #Time that the note started been played

            elif msg.type == 'note_off':
                if len(all_notes[note][-1]) != 1:
                    continue
                all_notes[note][-1].append(abs_time * SPM / TPM) #Time that the note stops

    ####### It needs an upgrade #######
    #Stopping any note that hasnt ended
    for note in all_notes:
        for start_end in all_notes[note]:
            if len(start_end) == 1:
                start_end.append(start_end[0] + 1)

    samples = []
    for note in all_notes:
        for start, end in all_notes[note]:
            sample_ix = int(start/SPM)
#             print(sample_ix)
            
            # fill in silence until the appropriate sample/measure is reached
            while len(samples) <= sample_ix:
                samples.append(np.zeros((SPM, 128), dtype=np.uint8))
#             print(samples)
            
            # get sample and find its start to encode the start of the note
            sample = samples[sample_ix]
            start_ix = int(start - sample_ix * SPM)
            sample[start_ix, note] = 1

    return samples

In [8]:
samp = midi_to_samples(mid[0], 48)

In [9]:
def samples_to_midi(samples, fname, SPM, thresh=0.5):
    #Creates mid object and adding track to it
    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)

    TPM = mid.ticks_per_beat #Get ticks per measure
    print(TPM)
    ticks_per_sample = TPM / SPM

    abs_time = 0
    last_time = 0

    for sample in samples:
        for y in range(sample.shape[0]):
            abs_time += ticks_per_sample

            for x in range(sample.shape[1]):
                note = x


                if sample[y,x] >= thresh and (y == 0 or sample[y - 1, x] < thresh):
                    delta_time = abs_time - last_time
                    track.append(mido.Message('note_on', note= note, velocity=64, time= int(delta_time)))
                    last_time = abs_time
                    
                if sample[y,x] >= thresh and (y == sample.shape[0] - 1 or sample[y + 1, x] < thresh):
                    delta_time = abs_time - last_time
                    track.append(mido.Message('note_off', note=int(note), velocity=64, time=int(delta_time)))
                    last_time = abs_time

    return mid.save(fname)


In [14]:
samples_to_midi(samp, "test720.mid",720)

480
